In [ ]:
import tensorflow as tf
print(tf.__version__)

In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn import metrics
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, BatchNormalization
from tensorflow.keras import optimizers
from keras.regularizers import l1, l2
import matplotlib.pyplot as plt
import joblib # To load or export models
import os

In [3]:
import warnings
warnings.filterwarnings('ignore')  # early-stop warnings

In [4]:
# Load the data for water content prediction
data = pd.read_csv("royal_city_daily_avg.csv") # Daily averages

# Split data into features and target variables
X = data[['wind_speed', 'solar_radiation', 'relative_humidity', 'air_temp', 'canopy_temp_mean', 'soil_temp']]
y_water_content = data['water_content_mean']

# Normalize the data
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Split data into training and testing sets
X_train_water_content, X_test_water_content, y_train_water_content, y_test_water_content = train_test_split(X_scaled, 
                                                                                                            y_water_content, 
                                                                                                            test_size=0.2, 
                                                                                                            random_state=42
                                                                                                            )

In [5]:
# Load the data for water potential prediction
data = pd.read_csv("royal_city_late_morning_avg.csv") # Late morning averages (9:00AM - 11:00PM)

# Split data into features and target variables
X = data[['wind_speed', 'solar_radiation', 'relative_humidity', 'air_temp', 'canopy_temp_mean', 'soil_temp']]
y_water_potential = data['water_potential_mean']

# Normalize the data
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Split data into training and testing sets
X_train_water_potential, X_test_water_potential, y_train_water_potential, y_test_water_potential = train_test_split(X_scaled, 
                                                                                                                    y_water_potential, 
                                                                                                                    test_size=0.2, 
                                                                                                                    random_state=42
                                                                                                                    )

In [ ]:
# Create a neural network model for water content prediction
model_nn_water_content = Sequential([
    Dense(100, activation='relu', input_dim=X_train_water_content.shape[1], kernel_regularizer=l2(0.01)), # Add regularization techniques like L1 or L2 regularization to prevent overfitting.
    # BatchNormalization(),
    Dense(100, activation='relu', kernel_regularizer=l2(0.01)),
    # BatchNormalization(),
    Dense(1, activation='linear')
])

# Optimizers: Adam, SGD, RMSprop, RMSprop, Adadelta
# Experiment with different learning rates (e.g., 0.01, 0.001, 0.0001) to find the optimal value.
model_nn_water_content.compile(loss="mse", optimizer=optimizers.Adam(learning_rate=0.0001))

model_nn_water_content.summary()

In [ ]:
model_nn_water_content.compile(optimizer='adam', loss='mean_squared_error')
model_nn_water_content.fit(X_train_water_content, y_train_water_content, epochs=100, batch_size=32, validation_split=0.2)

In [ ]:
# Evaluate the model
# Calculate Metrics
y_pred_water_content = model_nn_water_content.predict(X_test_water_content)
r2_nn_water_content = metrics.r2_score(y_test_water_content, y_pred_water_content)
mse_nn_water_content = metrics.mean_squared_error(y_test_water_content, y_pred_water_content)

# Update results dictionary
results_water_content = {}
results_water_content = {'MSE': mse_nn_water_content, 'R2': r2_nn_water_content, 'predicted_values': y_pred_water_content}

In [ ]:
# Create a neural network model for water potential prediction
model_nn_water_potential = Sequential([
    Dense(100, activation='tanh', input_dim=X_train_water_potential.shape[1], kernel_regularizer=l2(0.01)),  # Adjust hidden layer size
    # BatchNormalization(),
    Dense(50, activation='tanh', kernel_regularizer=l2(0.01)),
    # BatchNormalization(),
    Dense(1, activation='linear')
])

model_nn_water_potential.compile(loss="mse", optimizer=optimizers.SGD(learning_rate=0.001))  # Adjust optimizer and learning rate

model_nn_water_potential.summary()

In [ ]:

model_nn_water_potential.compile(optimizer='adam', loss='mean_squared_error')
model_nn_water_potential.fit(X_train_water_potential, y_train_water_potential, epochs=100, batch_size=32, validation_split=0.2)

In [ ]:
# Evaluate the model
# Calculate Metrics
y_pred_water_potential = model_nn_water_potential.predict(X_test_water_potential)
r2_nn_water_potential = metrics.r2_score(y_test_water_potential, y_pred_water_potential)
mse_nn_water_potential = metrics.mean_squared_error(y_test_water_potential, y_pred_water_potential)

# Update results dictionary
results_water_potential = {}
results_water_potential = {'MSE': mse_nn_water_potential, 'R2': r2_nn_water_potential, 'predicted_values': y_pred_water_potential}

In [ ]:
# Print results
print("Water Content Prediction Results:")
print(f"MSE={results_water_content['MSE']:.4f}, R2={results_water_content['R2']:.4f}")

print("\nWater Potential Prediction Results:")
print(f"MSE={results_water_potential['MSE']:.4f}, R2={results_water_potential['R2']:.4f}")

In [ ]:
# Visualize results for water content prediction (scatter plots of predicted vs. actual values)
fig, axs = plt.subplots(nrows=1, ncols=2, figsize=(12, 6))  # Create two subplots
model = 'TensorFlow Neural Network'

# Plot water content prediction
axs[0].scatter(y_test_water_content, y_pred_water_content, label=model)
axs[0].set_xlabel("Actual Water Content")
axs[0].set_ylabel("Predicted Water Content")
axs[0].set_title("Predicted vs. Actual Water Content")
axs[0].legend()

# Plot water potential prediction
axs[1].scatter(y_test_water_potential, y_pred_water_potential, label=model)
axs[1].set_xlabel("Actual Water Potential")
axs[1].set_ylabel("Predicted Water Potential")
axs[1].set_title("Predicted vs. Actual Water Potential")
axs[1].legend()

# Add text annotations with R² and MSE values
axs[0].text(0.55, 0.85, f"R²: {results_water_content['R2']:.4f}", ha='left', va='top', transform=axs[0].transAxes, fontsize=10)
axs[0].text(0.55, 0.80, f"MSE: {results_water_content['MSE']:.4f}", ha='left', va='top', transform=axs[0].transAxes, fontsize=10)

axs[1].text(0.55, 0.85, f"R²: {results_water_potential['R2']:.4f}", ha='left', va='top', transform=axs[1].transAxes, fontsize=10)
axs[1].text(0.55, 0.80, f"MSE: {results_water_potential['MSE']:.4f}", ha='left', va='top', transform=axs[1].transAxes, fontsize=10)

# Add diagonal line
for ax in axs:
    xlims = ax.get_xlim()
    ylims = ax.get_ylim()
    xmin, xmax = min(xlims), max(xlims)
    ymin, ymax = min(ylims), max(ylims)
    ax.set_xlim(xmin, xmax)
    ax.set_ylim(ymin, ymax)
    ax.plot(xlims, ylims, 'k--', alpha=0.75, zorder=0) # Adjust line style, alpha, and zorder as needed

plt.tight_layout()
plt.show()

In [14]:
# # Create the models folder if it doesn't exist
# model_folder = "models"
# if not os.path.exists(model_folder):
#     os.makedirs(model_folder)

# # Save the models (HDF5 format)
# model_nn_water_content.save(os.path.join(model_folder, "model_nn_water_content.h5"))
# model_nn_water_potential.save(os.path.join(model_folder, "model_nn_water_potential.h5")) 

In [15]:
# # Load the created neural network model
# model_nn_water_content = tf.keras.models.load_model(os.path.join(model_folder, "model_nn_water_content.h5"))
# model_nn_water_potential = tf.keras.models.load_model(os.path.join(model_folder, "model_nn_water_potential.h5"))